In [427]:
import pandas as pd
import numpy as np
import random

class DecisionTree:
    def __init__(self,depth=0,max_depth=8):
        #Read the data from csv file and name the columns
        
        c=['White King file (column)','White King rank (row)','White Rook file','White Rook rank','Black King file','Black King rank','Output']
        self.df=pd.read_csv('550-p1-cset-krk-1.csv',header=None)
        self.df=self.df.rename({0:'White King file (column)',1:'White King rank (row)',2:'White Rook file',3:'White Rook rank',4:'Black King file',5:'Black King rank',6:'Output'}, axis=1)
        df0,df1,df2,df3,df4,df5=self.processing_data(self.df)
        self.label_output()
        self.df=pd.concat([df0,df1,df2,df3,df4,df5,self.df['Output']],axis=1)
        self.left=None
        self.right=None
        self.fkey=None
        self.fval=None
        self.depth=depth
        self.max_depth=max_depth
        self.target=None
        self.d1={17:'draw',0:'zero',1:'one',2:'two',3:'three',4:'four',5:'five',6:'six',7:'seven',8:'eight',9:'nine',10:'ten',11:'eleven',12:'twelve',13:'thirteen',14:'fourteen',15:'fifteen',16:'sixteen'}

    def label_output(self):
        self.d={'draw':17,'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9,'ten':10,'eleven':11
          ,'twelve':12,'thirteen':13,'fourteen':14,'fifteen':15,'sixteen':16}
        for column in self.df:
            if column=='Output':
                s1=self.df[column].values
                for j,i in enumerate(s1):
                    s1[j]=self.d[i]
                break
        self.df=self.df.assign(Output=s1,inplace='True')

        
        
    def processing_data(self,data):
        # Labeling each data to 0-1, converting categorical to numerical data
        
        columns_text_0=['WKa','WKb','WKc','WKd','WKe','WKf','WKg','WKh']
        columns_data_0=['WK1','WK2','WK3','WK4','WK5','WK6','WK7','WK8']
        columns_text_1=['WRa','WRb','WRc','WRd','WRe','WRf','WRg','WRh']
        columns_data_1=['WR1','WR2','WR3','WR4','WR5','WR6','WR7','WR8']
        columns_text_2=['BKa','BKb','BKc','BKd','BKe','BKf','BKg','BKh']
        columns_data_2=['BK1','BK2','BK3','BK4','BK5','BK6','BK7','BK8']
        index=0
        for i in ['White King file (column)','White King rank (row)','White Rook file','White Rook rank','Black King file','Black King rank']:
            alphabets=[]
            numericals=[]
            for columndata in data[i]:
                letter=[0]*8
                numbers=[0]*8
                if not isinstance(columndata, int):
                    letter[ord(columndata)-ord('a')]=1
                    alphabets.append(letter)
                else:
                    numbers[ord(str(columndata))-ord('0')-1]=1
                    numericals.append(numbers)
            if index==0:
                df0=pd.DataFrame(data=alphabets, columns=columns_text_0)
            if index==1:
                df1=pd.DataFrame(data=numericals, columns=columns_data_0)
            if index==2:
                df2=pd.DataFrame(data=alphabets, columns=columns_text_1)
            if index==3:
                df3=pd.DataFrame(data=numericals, columns=columns_data_1)
            if index==4:
                df4=pd.DataFrame(data=alphabets, columns=columns_text_2)
            if index==5:
                df5=pd.DataFrame(data=numericals, columns=columns_data_2)
            index+=1
        return (df0,df1,df2,df3,df4,df5)
    
    def entropy(self,col):
        counts=np.unique(col,return_counts=True)
        ent=0.0
        for i in counts[1]:
            p=i/col.shape[0]
            ent+=(-1.0*p*np.log2(p))
        return ent
    
    def information_gain(self,x_data,fkey,fval):
        right,left=self.divide_data(x_data,fkey,fval)
        l=float(left.shape[0])/x_data.shape[0]
        r=float(right.shape[0])/x_data.shape[0]
        if left.shape[0]==0 or right.shape[0]==0:
            return float("-inf")
        i_gain=self.entropy(x_data.Output)-(l*self.entropy(left.Output)+r*self.entropy(right.Output))
        return i_gain
    
    def divide_data(self,x_data,fkey,fval):
        
        #fkey: Feature names 
        #fval: 
        
        x_right=pd.DataFrame([],columns=x_data.columns)
        x_left=pd.DataFrame([],columns=x_data.columns)
        for i in range(x_data.shape[0]):
            val = x_data[fkey].loc[i]
            if val >= fval:
                x_right = x_right.append(x_data.iloc[i])
            else:
                x_left = x_left.append(x_data.iloc[i])
        return x_right,x_left
    
    def frequency_of_Output(self, x_train):
        
        self.dict={}
        for i in x_train:
            if i not in self.dict:
                self.dict[i]=1
            else:
                self.dict[i]+=1
        return max(self.dict, key= lambda d: self.dict[d])
        
    def train(self,x_train):
        features=self.df.columns[:-1]
        info_gains=[]
        for i in features:
            i_gain=self.information_gain(x_train,i,0.5)
            info_gains.append(i_gain)
        self.fkey=features[np.argmax(info_gains)]
        self.fval=0.5
        print("Splitting Tree",self.fkey,"entropy",max(info_gains))
        data_right,data_left=self.divide_data(x_train,self.fkey,self.fval)
        data_right=data_right.reset_index(drop=True)
        data_left=data_left.reset_index(drop=True)
        if data_left.shape[0]==0 or data_right.shape[0]==0:
            self.target=self.d1[self.frequency_of_Output(x_train.Output)]
            return 
        if self.depth>=self.max_depth:
            
            self.target=self.d1[self.frequency_of_Output(x_train.Output)]
            return 
        self.left=DecisionTree(self.depth+1,self.max_depth)
        self.left.train(data_left)
        self.right=DecisionTree(self.depth+1,self.max_depth)
        self.right.train(data_right)

        self.target=self.d1[self.frequency_of_Output(x_train.Output)]
        return 
    
    def predict(self,test):
        if test[self.fkey] > self.fval:
            if self.right is None:
                return self.target
            return self.right.predict(test)
        if test[self.fkey] <= self.fval:
            if self.left is None:
                return self.target
            return self.left.predict(test)
    def dataframe(self):
        return self.df

    

        
#Creating Object of Decision Tree
d=DecisionTree()



# Splitting Data Into training, test and validate :60,20,20
train_data, validate_data, test_data = np.split(d.dataframe().sample(frac=1,random_state=42), [int(.6*len(d.dataframe())), int(.8*len(d.dataframe()))])

#Reset Index to 0
train_data=train_data.reset_index(drop=True)
test_data=test_data.reset_index(drop=True)

# Building tree
d.train(train_data)

Splitting Tree BK1 entropy 0.29434547342867523
Splitting Tree BKa entropy 0.32327386382137524
Splitting Tree BKb entropy 0.4729844284889768
Splitting Tree BK2 entropy 0.396641067962866
Splitting Tree BKh entropy 0.5487949406953987
Splitting Tree WKa entropy 0.0
Splitting Tree WKb entropy 0.0
Splitting Tree WKc entropy 0.0
Splitting Tree WK3 entropy 0.0
Splitting Tree WK1 entropy 0.0
Splitting Tree WK1 entropy 0.0
Splitting Tree WKa entropy -inf
Splitting Tree WRd entropy 0.0
Splitting Tree WKa entropy -inf
Splitting Tree WKc entropy 0.46691718668869964
Splitting Tree WRe entropy 0.650022421648354
Splitting Tree WK4 entropy 0.4199730940219749
Splitting Tree BK8 entropy 0.9182958340544896
Splitting Tree WR3 entropy 0.0
Splitting Tree WKa entropy -inf
Splitting Tree WK2 entropy 0.0
Splitting Tree WKa entropy -inf
Splitting Tree WKa entropy -inf
Splitting Tree WR3 entropy 0.7303078961588816
Splitting Tree WKc entropy 0.3788788371352292
Splitting Tree WRe entropy 0.9182958340544896
Splittin

In [442]:
def bagging_replacement(t_set, holdout_set,d):
    final_t_set = []
    final_holdout_set = []
    
    Training_indexes = list(t_set.index)
    Testing_indexes = list(holdout_set.index)
    union_set=Training_indexes+Testing_indexes
    union_set.sort()
    
    incorrect_array=accuracy(d,holdout_set)[1]


    for i in incorrect_array:
        union_set.append(i)
        union_set.append(i)

    for _ in range(len(t_set)):
        add_index = random.randint(0, len(t_set) - 1)
        final_t_set.append(union_set[add_index])



    # remove duplicates before removing items in final_t_set
    for item in union_set:
        if item not in final_holdout_set:
            final_holdout_set.append(item)
    
    for item in final_t_set:
        if item in final_holdout_set:
            final_holdout_set.remove(item)


    return final_t_set, final_holdout_set

def accuracy(d,test_data):

    count=0
    incorrect=[]
    correct=[]
    old_data=test_data.index

    test_data=test_data.reset_index(drop=True)
    y_pred=[]

    for i in range(test_data.shape[0]):
        y_pred.append(d.predict(test_data.loc[i]))


    for i in range(len(y_pred)):
        if y_pred[i]== d.d1[test_data['Output'][i]]:
            count+=1
            correct.append(i)
        else:
            incorrect.append(i)
    

        
    
    new_data=[]
    for i in incorrect:
        new_data.append(old_data[i])  
    return count/len(test_data),new_data


print("Accuracy of 1st DTree:",accuracy(d,test_data)[0]*100,"%")
train_data, validate_data, test_data = np.split(d.dataframe().sample(frac=1,random_state=42), [int(.6*len(d.dataframe())), int(.8*len(d.dataframe()))])
Training_Set, Holdout_Set = bagging_replacement(train_data, test_data,d)

Accuracy of 1st DTree: 50.0 %


In [443]:
def convert_indices_to_DataFrame(Training_Set,d):
    index1=[]
    Training_Set.sort()   
    d1=[]
    for i, j in d.dataframe().iterrows():
        if i in Training_Set:
            c1=Training_Set.count(i)
            for _ in range(c1):
                d1.append(d.dataframe()[i:i+1].values)
    v1=[]
    for i in d1:
        b1=[]
        for t in i:
            for r in t:
                b1.append(r)
        v1.append(b1)
    return v1


#d1=DecisionTree()
Training_Set_d2 = pd.DataFrame(data= convert_indices_to_DataFrame(Training_Set,d),columns=d.dataframe().columns)
HoldOut_Set_d2 = pd.DataFrame(data=  convert_indices_to_DataFrame(Holdout_Set,d),columns=d.dataframe().columns)
d.train(Training_Set_d2)

Splitting Tree WKc entropy 0.5105729873862126
Splitting Tree BK1 entropy 0.34209469252970637
Splitting Tree BKa entropy 0.6411187446397535
Splitting Tree BKb entropy 0.7793498372920851
Splitting Tree BK3 entropy 0.18639695711595625
Splitting Tree WKa entropy 0.0
Splitting Tree WKb entropy 0.0
Splitting Tree WK1 entropy 0.0
Splitting Tree WK3 entropy 0.0
Splitting Tree BK2 entropy 0.0
Splitting Tree WK1 entropy 0.0
Splitting Tree WKa entropy -inf
Splitting Tree WKa entropy -inf
Splitting Tree WR4 entropy 0.0
Splitting Tree WKa entropy -inf
Splitting Tree WKa entropy -inf
Splitting Tree WK3 entropy 1.0
Splitting Tree WKa entropy -inf
Splitting Tree WKa entropy -inf
Splitting Tree WRg entropy 0.9182958340544896
Splitting Tree WK1 entropy 0.0
Splitting Tree WKa entropy -inf
Splitting Tree WKa entropy -inf
Splitting Tree WKa entropy -inf
Splitting Tree WR5 entropy 0.5156849143255189
Splitting Tree WRc entropy 0.5916727785823275
Splitting Tree WR6 entropy 0.4204484631347318
Splitting Tree WK

In [444]:
print("Accuracy of 2nd DTree:",accuracy(d,HoldOut_Set_d2 )[0]*100,"%")

Accuracy of 2nd DTree: 28.723404255319153 %


In [415]:
import random
import math

# Each tree node consists of:
#   1. a vector list => [([feature vector], class), ..]
#   2. the selected attribute to split the vector list
#   3. its children => {selected attribute value : node, ..}
class node:
    def __init__(self):
        self.vecs = list()
        self.attr_split = None
        self.children = dict()


# function to read data file
# parameter: data file name or path to data file
# return: list of featre vectors and its corresponding class pair => [([feature_vector], class)..]
def read_file(filename):
    fea_vecs = []
    file = open(filename, "r")
    for line in file:
        values = line.split(",")
        fea_vecs.append((values[:6], values[6].replace("\n", "")))
    return fea_vecs


feature_vecs = []
attributes = ["White King file", " White King rank", "White Rook file", "White Rook rank", "Black King file", "Black King rank"]
feature_vecs = read_file("550-p1-cset-krk-1.csv")

Training_Set = []
Holdout_Set= []
Validation_Set = []


def sets_generator():
    total_data = len(feature_vecs)
    data_indexes = list(range(total_data))

    total_training_set_data = int(total_data * (.60));
    remaining_data = total_data - total_training_set_data

    if remaining_data % 2 == 1:
        total_training_set_data += 1
        remaining_data - 1

    Training_indexes = random.sample(data_indexes, total_training_set_data)

    for element in Training_indexes:
        Training_Set.append(feature_vecs[element])
        data_indexes.remove(element)


    Holdout_indexes = random.sample(data_indexes, int(remaining_data/2))

    for element in Holdout_indexes:
        Holdout_Set.append(feature_vecs[element])
        data_indexes.remove(element);


    Validation_indexes = list(data_indexes);

    for element in Validation_indexes:
        Validation_Set.append(feature_vecs[element])
        data_indexes.remove(element)


def entropy(class_occurrence, total):
    node_entropy = 0
    for minwindepth in class_occurrence:
        class_probability = class_occurrence[minwindepth] / total
        node_entropy += -class_probability*math.log(class_probability,2)
    return node_entropy


# Formula to find information gain
def FindInfoGain(entropy_set,entropy_attr,probab):
    return entropy_set-sum([entropy_attr[i]*probab[i] for i in range(len(entropy_attr))])


# Select best attribut to split a node
# Take as input a tree node
# When finish, tree node's property is modified accordingly
def split_node(root, attrs):
    # Calculate root entropy
    class_occurrence = dict()
    for pair in root.vecs:
        if pair[1] in class_occurrence:
            class_occurrence[pair[1]] += 1
        else:
            class_occurrence[pair[1]] = 1
    total = len(root.vecs)
    root_ent = entropy(class_occurrence, total)
    print("root entropy:", root_ent)

    # Calculate average entropy for each attribute value
    info_gain = list()
    for i in range(len(root.vecs[0][0])):
        class_occurence = dict()
        total = dict()
        for pair in root.vecs:
            if pair[0][i] not in class_occurence:
                class_occurence[pair[0][i]] = dict()
                total[pair[0][i]] = 1
            else:
                total[pair[0][i]] += 1
            if pair[1] in class_occurence[pair[0][i]]:
                class_occurence[pair[0][i]][pair[1]] += 1
            else:
                class_occurence[pair[0][i]][pair[1]] = 1
        entropy_list = list()
        for attr_val in class_occurence:
            entropy_list.append((total[attr_val], entropy(class_occurence[attr_val], total[attr_val])))
        avg_ent = 0
        for ent in entropy_list:
            avg_ent += ent[0] / len(root.vecs) * ent[1]
        info_gain.append(avg_ent)
        print("Attribute " + attrs[i] + "'s average entropy:", avg_ent)

    # Calculate information gain for each attribute
    print()
    max_info = (0, 0)
    for i in range(len(info_gain)):
        info_gain[i] = root_ent - info_gain[i]
        if info_gain[i] > max_info[1]:
            max_info = (i, info_gain[i])
        print("Attribute " + attrs[i] + "'s information gain:", info_gain[i])
    print("Attribute " + attrs[max_info[0]] + " has the greatest information gain, so it is selected as the attribute to split\n")

    # check if the selected attribute value produce the same size child
    same_attr_val = True
    max_info_index = max_info[0]
    while same_attr_val:
        val = root.vecs[0][0][max_info_index]
        for pair in root.vecs:
            if pair[0][max_info_index] != val:
                same_attr_val = False
        if same_attr_val:
            print("\n\n", max_info_index, val, root.vecs, "\n\n")
            max_info_index += 1

    # Updatte root node properties
    root.attr_split = max_info_index
    for pair in root.vecs:
        if pair[0][root.attr_split] not in root.children:
            root.children[pair[0][root.attr_split]] = node()
        root.children[pair[0][root.attr_split]].vecs.append(pair)

# check whether a node is a leaf node
def isLeaf(root):
    pair1 = root.vecs[0][1]
    for pair in root.vecs:
        if pair[1] != pair1:
            return False
    return True

# build a decision tree using the provided root node
def buildDTree(root):
    queue = [root]
    while len(queue) > 0:
        curr = queue.pop(0)
        split_node(curr, attributes)
        for child in curr.children:
            if not isLeaf(curr.children[child]):
                queue.append(curr.children[child])

# classified the provided vector using the provided rooted tree
def classifier(root, feat_vec):
    curr = root
    while not isLeaf(curr):
        if feat_vec[0][curr.attr_split] in curr.children:
            curr = curr.children[feat_vec[0][curr.attr_split]]
        else:
            class_occurence = dict()
            for pair in curr.vecs:
                if pair[1] in class_occurence:
                    class_occurence[pair[1]] += 1
                else:
                    class_occurence[pair[1]] = 1
            max = (0, "")
            for c in class_occurence:
                if class_occurence[c] > max[0]:
                    max = (class_occurence[c], c)
            return max[1]

    return curr.vecs[0][1]


def accuracy(tree_root, v_set):
    correct = 0;
    incorrect_indexes = []
    i = 0
    for pair in v_set:
        if (classifier(tree_root, pair) == pair[1]):
            correct += 1
        else:
            incorrect_indexes.append(i)
        i += 1
    return correct / len(v_set), incorrect_indexes


def bagging_replacement(root, t_set, holdout_set):
    union_set = t_set + holdout_set
    final_t_set = []
    final_holdout_set = []

    for i in accuracy(root, holdout_set)[1]:
        union_set.append(holdout_set[i])
        union_set.append(holdout_set[i])

    for i in t_set:
        add_index = random.randint(0, len(t_set) - 1)
        final_t_set.append(union_set[add_index])

    # remove duplicates before removing items in final_t_set
    for item in union_set:
        if item not in final_holdout_set:
            final_holdout_set.append(item)

    for item in final_t_set:
        if item in final_holdout_set:
            final_holdout_set.remove(item)

    return final_t_set, final_holdout_set


'''
attributes = ["crust size", "shape", "filling size"]
root = node()
root.vecs = [(["big", "circle", "small"], "pos"),(["small", "circle", "small"], "pos"),(["big", "square", "small"], "neg"),(["big", "triangle", "small"], "neg"),(["big", "square", "big"], "pos"),(["small", "square", "small"], "neg"),(["small", "square", "big"], "pos"),(["big", "circle", "big"], "pos")]
print("\n\nTest Data:\n", root.vecs, "\n\nAttributes:\n", attributes, "\n")
split_node(root, attributes)
print("\nAfter splitting:")
for child in root.children:
    print("Attribute " + child + " has vector list:\n", root.children[child].vecs, "\n")
'''

attributes = ["White King file", " White King rank", "White Rook file", "White Rook rank", "Black King file", "Black King rank"]
feature_vecs = read_file("550-p1-cset-krk-1.csv")

sets_generator()
print(len(feature_vecs))
print(len(Training_Set))
print(len(Holdout_Set))
print(len(Validation_Set))

root = node()
root.vecs = Training_Set
if not isLeaf(root):
    buildDTree(root)

# print(classifier(root, Holdout_Set[5]))
print("Printing first dTree accuracy")
print(accuracy(root, Validation_Set)[0])

Training_Set, Holdout_Set = bagging_replacement(root, Training_Set, Holdout_Set)

root = node()
root.vecs = Training_Set
if not isLeaf(root):
    buildDTree(root)

print("Printing second dTree accuracy")
print(accuracy(root, Holdout_Set)[0])

220
132
44
44
root entropy: 3.400212225825798
Attribute White King file's average entropy: 2.9545979920386065
Attribute  White King rank's average entropy: 3.1220480535072963
Attribute White Rook file's average entropy: 2.808546852686658
Attribute White Rook rank's average entropy: 2.896690904667118
Attribute Black King file's average entropy: 2.5362307047156794
Attribute Black King rank's average entropy: 2.8839589855435412

Attribute White King file's information gain: 0.44561423378719134
Attribute  White King rank's information gain: 0.27816417231850155
Attribute White Rook file's information gain: 0.5916653731391399
Attribute White Rook rank's information gain: 0.5035213211586798
Attribute Black King file's information gain: 0.8639815211101185
Attribute Black King rank's information gain: 0.5162532402822566
Attribute Black King file has the greatest information gain, so it is selected as the attribute to split

root entropy: 2.005977711423774
Attribute White King file's average ent